<a href="https://colab.research.google.com/github/Midhilesh4890/Large-Language-models-practice/blob/main/Mercor_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!pip install torch torchvision transformers python-dotenv
!pip install huggingface_hub --upgrade
!apt-get install wget coreutils
!pip install requests

In [ ]:
!git clone https://github.com/meta-llama/llama3.git

In [ ]:
%cd llama3
!pip install -r requirements.txt

In [ ]:
!chmod +x download.sh

In [ ]:
# Store the URL and model name in a variable
url = "https://download6.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoiOXhsbTlhd3czbjAwNDJxc3NsbjFyNmIzIiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZG93bmxvYWQ2LmxsYW1hbWV0YS5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3MTY5MjEzMDJ9fX1dfQ__&Signature=E%7E6anzNIlvIol5R8JXT4jLP0rcIXkKPg%7E6jEFwzEr3tGMAyK9LCOtvXHlUGUO0pOCRf3-Jw%7EdIl3j30lY%7ELYmWP77NnoyLKIJDcQgRByi01vblVy3OnoQSyFzA-M5fAhPfzJ%7EH7v2aLLo433%7ErqIKEu1mnYVee%7EDeU%7EytJbMNsjHp3BTBzvrJ98e89y%7EtMQB9iXr5acMxoOj19WG4RReworhYSO7HnTWfDvAsOfSwEjgY6EpUJ2AVpfxo2H5EDookDFOdeT4EfCbhMSGDE-sJeBp6ERwJwFAzBmEHRKgKdfZ0o-PKm0gC0qjctWGCsJfSSXPFSWPjZ%7EGsJ25VB7rWQ__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=545877497846408"
model_name = "8B,8B-instruct"

# Download the model using the !download.sh script
!./download.sh "$url" "{model_name}"

In [ ]:
# import os
# from dotenv import load_dotenv
# from huggingface_hub import HfApi, notebook_login

# load_dotenv(".env")
# HFTOKEN = os.environ.get("HFTOKEN")
# notebook_login(HFTOKEN)
# !git clone https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct /content/meta-llama/Meta-Llama-3-8B-Instruct

In [ ]:
!ls /content/llama3

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !mv /content/meta-llama/Meta-Llama-3-8B-Instruct /content/drive/LLAMA3/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [ ]:
# class RMSNorm(nn.Module):
#     def __init__(self, d_model, eps=1e-8):
#         super(RMSNorm, self).__init__()
#         self.eps = eps
#         self.scale = nn.Parameter(torch.ones(d_model))

#     def forward(self, x):
#         rms = torch.sqrt(torch.mean(x ** 2, dim=-1, keepdim=True) + self.eps)
#         logging.basicConfig(filename='rmsnorm.log', level=logging.INFO)
#         logging.info(f"RMSNorm value: {rms.mean().item()}")
#         return x * self.scale / rms

# Define RMSNorm class with logging
class RMSNorm(torch.nn.Module):
    def __init__(self, dim, layer_name, eps=1e-8):
        super().__init__()
        self.eps = eps
        self.scale = torch.nn.Parameter(torch.ones(dim))
        self.layer_name = layer_name
        self.call_count = 0

    def forward(self, x):
        self.call_count += 1
        if self.call_count <= 65:
            rms = torch.sqrt(torch.mean(x**2, dim=-1, keepdim=True) + self.eps)
            logging.info(f'Layer: {self.layer_name}, Call: {self.call_count}, RMS: {rms}')
        return x / rms * self.scale

In [ ]:
import torch
import os
from transformers import AutoTokenizer

In [ ]:
# Correct the import path to the model
from llama import LlamaForCausalLM

def load_pretrained_model(model_name='llama-3-8B'):
    model = LlamaModel.from_pretrained(model_name)
    # Replace existing normalization with the modified RMSNorm if needed
    for name, module in model.named_modules():
        if isinstance(module, nn.LayerNorm):  # Assuming LayerNorm needs to be replaced
            setattr(model, name, RMSNorm(module.normalized_shape[0]))
    return model

model = load_pretrained_model()
model.to(torch.float16)  # Set model to BF16 precision

In [ ]:
def evaluate_model(model, dataset):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        # Example: loop through the dataset and compute accuracy
        for inputs, targets in dataset:
            predictions = model(inputs)
            # Compute accuracy logic here
    return accuracy

# Assuming dataset loading and preparation is done
accuracy = evaluate_model(model, dataset)
print(f"Model accuracy: {accuracy}")


In [ ]:
# Load logged RMS values
rms_values = np.loadtxt('rmsnorm.log', usecols=[-1])

# Plot histogram
sns.histplot(rms_values, kde=True)
plt.title("Histogram of RMSNorm values")
plt.xlabel("RMSNorm Value")
plt.ylabel("Frequency")
plt.show()
